In [217]:
import pandas as pd

# First dataframe
data1 = {'C1': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
         'C2': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']}
df = pd.DataFrame(data1, index=range(1, 11))

# Second dataframe
data2 = {'C3': ['a', 'e']}
df2 = pd.DataFrame(data2, index=[1, 5])
df2_saved_index = df2.index


## The Amazing Index in Pandas

Here we have two data frames. How do we combine them neatly with the help of index?  

1. `merge`: and explicitly set left_index and right_index to be True
2. `concat`: index automatically matched
    * What if I reorder df2, would it mess things up? 😝
    * No it would be fine 💃🏻
3. Just assign the values directly, and the assignment is done by index!😎

At the end there are also two bonus sections
* What will happen if we use the above methods on two dataframes with no matching index? (spoiler alert: things are still neat 🥂)
* Finally, are there anything we can do to mess things up? 🤪 Yes, pls see at the end.

### 1. Merge

In [218]:
# Merge the two dataframes, explicitly based on the index
merged_df = df2.merge(df, left_index=True, right_index=True)

# Display the merged dataframe
print(merged_df)

  C3  C1 C2
1  a  10  A
5  e  50  E


### 2. Concat

In [219]:
# Didn't specify but notice that the rows were matched by index automatically
concatenated_df = pd.concat([df2, df], axis=1, join='inner')
print(concatenated_df)

  C3  C1 C2
1  a  10  A
5  e  50  E


In [220]:
# reorder df2, but notice the index is unchanged
df2.sort_values('C3', ascending = False, inplace = True)
print(df2)


  C3
5  e
1  a


In [221]:
pd.concat([df2, df], axis = 1, join = 'inner')

,C3,C1,C2
5,e,50,E
1,a,10,A


## 3. Assign directly

In [222]:
# Even if we simply assign the values like below, the assignment were based on index
df2['C1'] = df['C1']
df2['C2'] = df['C2']

print(df2)


  C3  C1 C2
5  e  50  E
1  a  10  A


## Mess Things Up

So even if we reorder the dataframe, the index will stay in place and help with matching. What could we do to mess things up?

Remember df2 has been reordered by C3 above, but the index are still the original index? What if we use `set_index()` and feed a saved index into it?

In [223]:
df2.set_index(df2_saved_index)

,C3,C1,C2
1,e,50,E
5,a,10,A


Now finally the index are no longer the original one but has been reset. If we do concat again now, it will be done using the newly assigned index and no longer consistent with the original. Pls see below. 

In [224]:
pd.concat([df2.set_index(df2_saved_index), df], axis = 1, join = 'inner')

,C3,C1,C2,C1,C2
1,e,50,E,10,A
5,a,10,A,50,E


# What if the index are not matched?

In [225]:
# recall the df has index ranging from 1 to 11
df.index

RangeIndex(start=1, stop=11, step=1)

In [226]:
# create a 3rd dataframe with index 21 and 25
data3 = {'C3': ['aa', 'ee']}
df3 = pd.DataFrame(data3, index=[21, 25])


In [227]:
# merge result in an empty dataframe as there is no matching index
df.merge(df3, right_index=True, left_index=True)

,C1,C2,C3


In [228]:
# pd.concat will stack two dataframes by index
pd.concat([df,df3], axis=1)

,C1,C2,C3
1,10.0,A,NaN
2,20.0,B,NaN
3,30.0,C,NaN
4,40.0,D,NaN
5,50.0,E,NaN
6,60.0,F,NaN
7,70.0,G,NaN
8,80.0,H,NaN
9,90.0,I,NaN
10,100.0,J,NaN


In [229]:
# because there is no matching index, the assigned C2 column will have missing values
df3['C2'] = df['C2']
df3

,C3,C2
21,aa,NaN
25,ee,NaN


## Quick note: Filter by location

The index is convenient in many ways. Say I can easily filter the target `y` and train_features `X` if I make sure the index is properly matched. 

In [230]:
# create one pandas df and one series, note that the index is assigned automatically ranging from 0 to 4
# or in many real-world cases, X and y would've come from a same df so sharing the index 
X = pd.DataFrame(range(1,6,1), columns=['value'])
y = pd.Series(range(6,11,1))

print("X:", X)
print("y:", y)

X:    value
0      1
1      2
2      3
3      4
4      5
y: 0     6
1     7
2     8
3     9
4    10
dtype: int64


In [231]:
# natually we can filter X like below
filtered_x = X[X['value']>3]
print(filtered_x)

   value
3      4
4      5


In [232]:
# due to the shared index, we can filter y just the same 😎
filtered_y = y[X['value']>3]
print(filtered_y)

3     9
4    10
dtype: int64


In [233]:
import numpy as np
# Create a dummy DataFrame with infinite values
df = pd.DataFrame({'col1': [np.nan, 2, np.inf, 4, 8], 
                   'col2': [6, 7, 8, np.inf, np.nan], 'target': [0,1,0,1,0]})
print(df)
# like many real-world data, this df have a specified index so probably easier to use iloc to identify records
df.index = list(range(1999, # start
                      1999 + df.shape[0], # end
                      1 # step
                      ))

X = df.drop('target', axis = 1)
y = df['target']

# find the coordinates (row, col) with infinite values 
infinite_indices = np.where(np.isinf(X))
print('rows location index with infinit values:', infinite_indices[0])
print('location index of infinite values (row, col)')
print([f'({row}, {col})' for row, col in zip(infinite_indices[0], infinite_indices[1])])

# get the location list of all the records/rows
all = list(range(df.shape[0]))
# use set to exclude those with infinite values, don't forget to turn it back to a list
rows_without_infinit_values = list(set(all) - set(list(infinite_indices[0])))
# the very cool thing is we can easily filter for both X and y this way so they are still neatly matched
X2 = X.iloc[rows_without_infinit_values]
y2 = y.iloc[rows_without_infinit_values]

   col1  col2  target
0   NaN   6.0       0
1   2.0   7.0       1
2   inf   8.0       0
3   4.0   inf       1
4   8.0   NaN       0
rows location index with infinit values: [2 3]
location index of infinite values (row, col)
['(2, 0)', '(3, 1)']


Alternatively we can get row and col indices respectively from `np.where()`. The row and col are very helpful for filtering. However if we want to manipulate or treat the specific value of interests (say replace with certain value) then the combined row-col-index is more useful. 

In [234]:
# row and col index directly
row_index, col_index = np.where(df.isna())
print('null value row location index', row_index)
print('null value col location index', col_index)

null value row location index [0 4]
null value col location index [0 1]


### A note of causion
To ensure the index array works as expected, it is always prudent to do a quick test on a simple dummy dataset first. Because the indices produced are tuple of arrays representing row and column indices, and may not match the dimensions and alignment of the dataframe of interest. For example, if we use the index to replace values, we are in for a surprise! 

In [235]:
# recall this is what the original df looks like
print(df)

      col1  col2  target
1999   NaN   6.0       0
2000   2.0   7.0       1
2001   inf   8.0       0
2002   4.0   inf       1
2003   8.0   NaN       0


In [236]:
# null_index through `np.where` and `isna()`
null_index = np.where(df.isna())
print(df.iloc[null_index])
# similarly print out indices with infinite values
infinite_indices = np.where(np.isinf(df))
print(df.iloc[infinite_indices])

      col1  col2
1999   NaN   6.0
2003   8.0   NaN
      col1  col2
2001   inf   8.0
2002   4.0   inf


In [237]:
# try use the index in `iloc[]` to locate value of interests and replace/label them
# we notice the iloc methods affected both the cols and rows involved
# whereas the replace is more precise in identifying the exact location of interests
df.iloc[null_index] = 'missing'
df.replace([-np.inf, np.inf], 'infinite', inplace = True)
print(df)

          col1      col2  target
1999   missing   missing       0
2000       2.0       7.0       1
2001  infinite       8.0       0
2002       4.0  infinite       1
2003   missing   missing       0
